In [1]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список silver S3

In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [3]:
bucket = s3.Bucket('silver')

In [4]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [5]:
objects[0], objects[1]

(s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet'))

In [6]:
total_size_mb = 0
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    total_size_mb += size_mb
    print(key, size_mb, obj.last_modified)

hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet 2.0 2025-12-12 09:04:06.609000+00:00
hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet 4.0 2025-12-12 09:04:38.370000+00:00
hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet 4.0 2025-12-12 09:05:07.196000+00:00
hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet 1.0 2025-12-12 09:05:16.560000+00:00
hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet 0.0 2025-12-12 09:05:23.998000+00:00
hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet 7.0 2025-12-12 09:06:17.416000+00:00
hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet 6.0 2025-12-12 09:07:02.054000+00:00
hh/vacancies/year=2025/month=11/day=19/part-1471860e-073d-4cdc-bb43-7ed3639cb269.parquet 6.0 2025-12-12 09:07:42.21600

In [7]:
print('Silver layer size', total_size_mb, 'mb')

Silver layer size 171.0 mb


In [11]:
# objects = [obj for obj in objects if silver_path2dt(obj).day >8 and silver_path2dt(obj).month == 12]
# objects

# Общее кол-во вакансий в S3

In [8]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet') 61
s3.ObjectSummary(bucket_name='silv

In [9]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [10]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
812632,812625


In [25]:
df_lazy.tail(1).collect().glimpse()

Rows: 1
Columns: 61
$ id                                    <str> '128392992'
$ name                                  <str> 'Бетонщик (Забайкальский край)'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'between1And3', 'name': 'От 1 года до 3 лет'}
$ address                        <struct[10]> {'city': 'посёлок городского типа Новая Чара', 'street': None, 'building': None, 'lat': 56.79906, 'lng': 118.27571, 'description': None, 'raw': 'посёлок городского типа Новая Чара', 'metro': None, 'metro_stations': [], 'id': '19428704'}
$ employer                        <struct[9]> {'id': '11875449', 'name': 'ПрессБук', 'url': 'https://api.hh.ru/em

# Lil EDA

In [11]:
objects[-1]

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=11/part-7808ee1e-f2b7-49f3-992b-722419096a95.parquet')

In [12]:
df_lazy = (
    # pl.scan_parquet([f"s3://silver/{obj.key}" for obj in objects], storage_options=storage_options)
    pl.scan_parquet(f"s3://silver/hh/vacancies/year=2025/month=12/day=11/part-7808ee1e-f2b7-49f3-992b-722419096a95.parquet", storage_options=storage_options)

    # Published Date transfromation from UTC to Local
    .with_columns(
        published_at_local = (
                pl.col('published_at_utc').dt.replace_time_zone(None) +
                pl.duration(minutes=pl.col('published_at_offset')
            )
        )
    )
    .with_columns(
        published_date=pl.col('published_at_local').dt.date()
    )
)

In [18]:
df_lazy.select('id').count().collect()

id
u32
123408


## Describe

In [13]:
df_lazy_described = df_lazy.describe()

In [14]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* classic notebook */
div.output_area pre {
  white-space: pre !important;   /* не переносить строки */
}

/* jupyterlab */
div.jp-OutputArea-output pre {
  white-space: pre !important;   /* не переносить строки */
}
</style>
"""))

with pl.Config(tbl_cols=-1, tbl_formatting='MARKDOWN', tbl_width_chars=-1):
    print(df_lazy_described)

shape: (9, 55)
| statistic  | id        | name                            | schedule | working_time_modes | working_time_intervals | working_days | working_hours | work_schedule_by_days | fly_in_fly_out_duration | is_adv_vacancy | internship | accept_temporary | accept_incomplete_resumes | premium  | has_test | show_contacts | response_letter_required | show_logo_in_search | archived | night_shifts | url                             | alternate_url                   | apply_alternate_url             | response_url                    | adv_response_url | immediate_redirect_url          | salary  | salary_range | employer | department | type     | employment | employment_form | experience | area     | address | work_format | relations | professional_roles | contacts | adv_context | sort_point_distance | snippet  | branding | insider_interview | video_vacancy | brand_snippet | date                | published_at_utc           | created_at_utc             | published_at_offset | created_at_o

## Check nulls

In [15]:
df_lazy.null_count().collect().glimpse()

Rows: 1
Columns: 54
$ id                        <u32> 0
$ name                      <u32> 0
$ schedule                  <u32> 0
$ working_time_modes        <u32> 0
$ working_time_intervals    <u32> 0
$ working_days              <u32> 0
$ working_hours             <u32> 0
$ work_schedule_by_days     <u32> 0
$ fly_in_fly_out_duration   <u32> 0
$ is_adv_vacancy            <u32> 0
$ internship                <u32> 0
$ accept_temporary          <u32> 0
$ accept_incomplete_resumes <u32> 0
$ premium                   <u32> 0
$ has_test                  <u32> 0
$ show_contacts             <u32> 0
$ response_letter_required  <u32> 0
$ show_logo_in_search       <u32> 69609
$ archived                  <u32> 0
$ night_shifts              <u32> 0
$ url                       <u32> 0
$ alternate_url             <u32> 0
$ apply_alternate_url       <u32> 0
$ response_url              <u32> 100478
$ adv_response_url          <u32> 123408
$ immediate_redirect_url    <u32> 100478
$ salary                 

## Слишком много null'ов в silver, проверка заполненности данных в bronze

In [19]:
bucket = s3.Bucket('bronze')

In [20]:
report_dt = "2025-12-11"
report_dt_objects = [o for o in bucket.objects.filter(Prefix=f"hh/vacancies/date={report_dt}")]
len(report_dt_objects)

25

In [21]:
for obj in report_dt_objects:
    s3path = f"s3://bronze/{obj.key}"
    lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)
    null_cnt = lazy_bronze.null_count().collect()
    print(null_cnt)
    

shape: (1, 49)
┌─────────────┬─────────────┬────────┬────────────┬───┬──────────┬─────┬─────────────┬─────────────┐
│ night_shift ┆ working_day ┆ salary ┆ created_at ┆ … ┆ has_test ┆ url ┆ brand_snipp ┆ immediate_r │
│ s           ┆ s           ┆ ---    ┆ ---        ┆   ┆ ---      ┆ --- ┆ et          ┆ edirect_url │
│ ---         ┆ ---         ┆ u32    ┆ u32        ┆   ┆ u32      ┆ u32 ┆ ---         ┆ ---         │
│ u32         ┆ u32         ┆        ┆            ┆   ┆          ┆     ┆ u32         ┆ u32         │
╞═════════════╪═════════════╪════════╪════════════╪═══╪══════════╪═════╪═════════════╪═════════════╡
│ 0           ┆ 0           ┆ 2232   ┆ 0          ┆ … ┆ 0        ┆ 0   ┆ 10608       ┆ 8917        │
└─────────────┴─────────────┴────────┴────────────┴───┴──────────┴─────┴─────────────┴─────────────┘
shape: (1, 49)
┌─────────────┬─────────────┬────────┬────────────┬───┬─────┬──────────┬─────────────┬─────────────┐
│ night_shift ┆ working_day ┆ salary ┆ created_at ┆ … ┆ url ┆

KeyboardInterrupt: 

In [22]:
obj = report_dt_objects[4]
s3path = f"s3://bronze/{obj.key}"
lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)

In [23]:
'description' in lazy_bronze.collect_schema().names()

False

In [30]:
silver_lazy = pl.scan_parquet(f"s3://silver/hh/vacancies/year=2025/month=12/day=11/part-7808ee1e-f2b7-49f3-992b-722419096a95.parquet", storage_options=storage_options)
silver_schema = set(silver_lazy.collect_schema().names())
bronze_schema = set(lazy_bronze.collect_schema().names())
len(silver_schema), len(bronze_schema)

(52, 49)

In [31]:
len(silver_schema & bronze_schema)

47

In [32]:
len(silver_schema - bronze_schema)

5

In [33]:
silver_schema - bronze_schema

{'created_at_offset',
 'created_at_utc',
 'date',
 'published_at_offset',
 'published_at_utc'}

In [34]:
silver_lazy.select('date').head(10).collect()

date
date
2025-12-11
2025-12-11
2025-12-11
2025-12-11
2025-12-11
2025-12-11
2025-12-11
2025-12-11
2025-12-11


In [29]:
'date' in silver_schema

True

## professional_role count

In [84]:
df_lazy.head(1).collect().glimpse()

Rows: 1
Columns: 63
$ id                                    <str> '107445022'
$ name                                  <str> 'Кладовщик'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'shift', 'name': 'Сменный график'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'noExperience', 'name': 'Нет опыта'}
$ address                        <struct[10]> null
$ employer                        <struct[9]> {'id': '1942330', 'name': 'Пятёрочка', 'url': 'https://api.hh.ru/employers/1942330', 'alternate_url': 'https://hh.ru/employer/1942330', 'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original-round/7101933.png', '90': 'https://img.hhcdn.ru/employer-logo-round/7101934.png', '240': 'https://img.hhcdn.ru/employer-logo-ro

In [92]:
q = (df_lazy.with_columns(
        pl.col('professional_roles').list.get(0).struct.field('name').alias('professional_role')
    )
)
q.select('professional_role').unique().count().collect()

professional_role
u32
174
